In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
df = pd.read_csv('selected_features.csv')

In [3]:
df

,reviewerID,reviewerFriendCount,reviewerNumReviews,reviewerFirstCount,reviewerUsefulCount,reviewerCoolCount,reviewerFunnyCount,reviewerComplimentCount,reviewerTipCount,reviewerFanCount,...,Num_Punc,verb_count,adj_count,polarity,reviewRating_1,reviewRating_2,clean_tokens,preprocessed,combined_text,flagged
0,bNYesZ944s6IJVowOnB0iA,0.002337,0.018008,0.018008,0.001885,0.000254,0.000279,0.000050,0.000000,0.001563,...,0.446875,0.776256,0.427350,1.391395e-01,0,0,"['unlike', 'next', 'eaten', 'previous', 'night...","['unlike', 'next', 'wed', 'eaten', 'previous',...","['the previous night', 'four hours and thirty-...",0
1,TRKxLC3y-ZvP45e5iilMtw,0.000000,0.001149,0.001149,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.031250,0.068493,0.068376,4.666667e-01,0,0,"['probably', 'one', 'best', 'meals', 'ever', '...","['probably', 'one', 'best', 'meal', 'ive', 'ev...","['an evening', 'a least 4 hours'] [] [] [] ['G...",0
2,0EMm8umAqXZzyhxNpL4M9g,0.000425,0.003831,0.003831,0.000276,0.000000,0.000056,0.000000,0.000000,0.000000,...,0.028125,0.063927,0.042735,4.133333e-01,0,0,"['service', 'impeccable', 'experience', 'prese...","['service', 'impeccable', 'experience', 'prese...","[] [] [] [] [] [] ['service', 'impeccable', 'e...",0
3,DlwexC7z88ymAzu45skODw,0.017850,0.022605,0.022605,0.001609,0.000406,0.000335,0.000150,0.005078,0.000000,...,0.262500,0.237443,0.290598,1.205482e-01,0,0,"['problem', 'places', 'like', 'given', 'exhorb...","['problem', 'place', 'like', 'given', 'exhorbi...","['5 seconds'] [] [] [] ['Jackson Pollock', 'Gr...",0
4,kW2dk1CWihmh3g7k9N2G8A,0.070761,0.462835,0.462835,0.075869,0.034472,0.023258,0.007082,0.010157,0.109375,...,0.090625,0.123288,0.170940,1.674603e-01,0,0,"['idea', 'write', 'review', 'dining', 'alinea'...","['idea', 'write', 'review', 'dining', 'alinea'...",['a few days'] [] ['American'] ['Chicago'] [] ...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26951,tivh8lr6pzBDNfrJLYWh_g,0.006587,0.062835,0.062835,0.009242,0.002843,0.000948,0.000601,0.000000,0.004687,...,0.050000,0.105023,0.051282,2.266234e-01,0,0,"['taco', 'bell', 'higher', 'prices', 'taco', '...","['taco', 'bell', 'higher', 'price', 'taco', 'b...","[] [] [] [] [] [] ['taco', 'bell', 'higher', '...",0
26952,jKs4FQgkV0wSX8BG2_dgTg,0.001275,0.003831,0.003831,0.000322,0.000102,0.000000,0.000025,0.000000,0.001563,...,0.053125,0.077626,0.076923,2.692708e-01,0,0,"['yellow', 'rose', 'favorite', 'mine', 'go', '...","['yellow', 'rose', 'favorite', 'mine', 'id', '...","['every day', 'Sundays'] [] [] [] [] [] ['yell...",0
26953,vX6aOMQ3HWCbwZVfCkCauw,0.002762,0.018391,0.018391,0.002667,0.000457,0.000558,0.000175,0.000462,0.001563,...,0.025000,0.027397,0.042735,-1.190476e-01,0,0,"['bad', 'ate', 'odd', 'wicker', 'seats', 'invi...","['bad', 'ate', 'odd', 'wicker', 'seat', 'invit...","[] [] [] [] [] [] ['bad', 'ate', 'odd', 'wicke...",0
26954,vX6aOMQ3HWCbwZVfCkCauw,0.002762,0.018391,0.018391,0.002667,0.000457,0.000558,0.000175,0.000462,0.001563,...,0.037500,0.068493,0.042735,6.938894e-18,0,0,"['surprised', 'eagerly', 'roped', 'pop', 'triv...","['surprised', 'eagerly', 'roped', 'pop', 'triv...","[] [] [] [] [] [] ['surprised', 'eagerly', 'ro...",0


In [4]:
df.columns

Index(['reviewerID', 'reviewerFriendCount', 'reviewerNumReviews',
       'reviewerFirstCount', 'reviewerUsefulCount', 'reviewerCoolCount',
       'reviewerFunnyCount', 'reviewerComplimentCount', 'reviewerTipCount',
       'reviewerFanCount', 'NRP_count', 'PERSON_count',
       'date_diff_joinedandreviewed', 'Num_Char', 'Num_Punc', 'verb_count',
       'adj_count', 'polarity', 'reviewRating_1', 'reviewRating_2',
       'clean_tokens', 'preprocessed', 'combined_text', 'flagged'],
      dtype='object')

Mock Train Test Split

In [15]:
from sklearn.model_selection import GroupShuffleSplit

# remove target variable
X = df.drop(['flagged'], axis=1) 

y = df.flagged

splitter = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=42)
split = splitter.split(X, groups=X['reviewerID'])
train_inds, test_inds = next(split)

X_train = X.iloc[train_inds]
X_test = X.iloc[test_inds]
y_train = y.iloc[train_inds]
y_test = y.iloc[test_inds]

# remove non-numerical variables from X sets
X_train = X_train.drop(['reviewerID', 'clean_tokens', 'preprocessed', 'combined_text'], axis=1)
X_test = X_test.drop(['reviewerID', 'clean_tokens', 'preprocessed', 'combined_text'], axis=1)

In [16]:
model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [17]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print('-----------------------------------------------------')
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print('-----------------------------------------------------')
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8276176577891281
-----------------------------------------------------
Confusion Matrix:
 [[4133  152]
 [ 793  404]]
-----------------------------------------------------
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.96      0.90      4285
           1       0.73      0.34      0.46      1197

    accuracy                           0.83      5482
   macro avg       0.78      0.65      0.68      5482
weighted avg       0.81      0.83      0.80      5482

